In [1]:
#Copied from /bin/basenji_sat_bed.py
from __future__ import print_function

from optparse import OptionParser

import gc
import json
import os
#os.environ['LD_LIBRARY_PATH'] = '/clusterfs/nilah/oberon/python/lib:/global/software/sl-7.x86_64/modules/langs/python/3.7/lib' #+ os.environ.get('LD_LIBRARY_PATH', '')
import pdb
import pickle
from queue import Queue
import random
import sys
from threading import Thread
from collections import defaultdict
import csv
import argparse
import os

import h5py
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf
from tqdm import tqdm

if tf.__version__[0] == '1':
  tf.compat.v1.enable_eager_execution()

from basenji import bed
from basenji import dna_io
from basenji import seqnn
from basenji import stream

#I added these
import pysam
import pyBigWig
import matplotlib.pyplot as plt
from scipy import signal
from Bio import motifs
from Bio.Seq import Seq
import itertools
from pyliftover import LiftOver
import pybedtools
import subprocess
from datetime import datetime as dt
import seaborn as sns
from pybedtools import BedTool
import pyBigWig

parser = argparse.ArgumentParser(description="This script runs through all the gene TSS's annotated in hg38 on the specified chromosome and provides their sequence, centered at the TSS.")

parser.add_argument("-c","--chromosome",help="chromosome to run",type=str)

args = parser.parse_args()

chromosome = args.chromosome


################################################################
# Specify filepaths
################################################################
targets_path = '/clusterfs/nilah/oberon/basenji/manuscripts/cross2020/targets_human.txt'
weights_path = '/clusterfs/nilah/oberon/basenji/manuscripts/cross2020/model_human.h5'
architecture_path = '/clusterfs/nilah/oberon/basenji/manuscripts/cross2020/params_human.json'

hg38_genome_path = '/clusterfs/nilah/ayesha/basenji/baselines/gm12878/data/hg38.fa'

#################################################################
# Load neural network
#################################################################
with open(architecture_path) as arch_open:
    arch = json.load(arch_open)
params_model = arch['model']
targets_df = pd.read_table(targets_path,index_col=0)
target_slice = targets_df.index
seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore(weights_path)
seqnn_model.build_slice(target_slice)

2023-11-16 14:18:24.453502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 14:18:28.718309: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /global/software/sl-7.x86_64/modules/langs/cuda/11.2/lib64:/global/home/groups/consultsw/sl-7.x86_64/modules/cudnn/7.0.5-cuda9.0/lib64:/global/software/sl-7.x86_64/modules/langs/python/3.7/lib
2023-11-16 14:18:28.718509: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open s

SystemExit: 2

/clusterfs/nilah/oberon/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
